In [ ]:
import random

def random_mat(n, seq = (0,1)):
    '''
    Create a random n by n matrix filled with elements from seq iterable
    '''
    A = [[random.choice(seq) for i in range(n)] for j in range(n)]
    return A

def print_mat(A):
    '''
    Helper function to print matrix
    '''
    for row in A:
        print(' '.join(map(str, row)))
    print()

def pad(A):
    '''
    add zeros column and row
    '''
    for row in A:
        row.append(0)
    A.append([0 for i in range(len(A[0]))])
    
    return A

def unpad(A):
    '''
    remove a column of zeros and ones
    '''
    for row in A:
        row.pop()
    A.pop()
    
    return A

def split(M):
    '''
    Helpler function to divide the submatrices
    '''
    n = len(M[0]) // 2
    
    A, B, C, D = [[[0 for i in range(n)] for i in range(n)] for i in range(4)]
    for i in range(n):
        for j in range(n):
            A[i][j] = M[i][j]
            B[i][j] = M[i][j + n]
            C[i][j] = M[i + n][j]
            D[i][j] = M[i + n][j + n]
    
    return A, B, C, D

def add(A, B, sign = 1):
    '''
    add (or subtract) two matrices
    '''
    # declare global variable to count the number of additions
    global ops
    
    n = len(A[0])
    C = [[0 for i in range(n)] for j in range(n)]
    
    if sign:
        for i in range(n):
            for j in range(n):
                C[i][j] = A[i][j] + B[i][j]
                ops += 1
    else:
        for i in range(n):
            for j in range(n):
                C[i][j] = A[i][j] - B[i][j]
                ops += 1
    
    return C

def merge(A, B, C, D):
    n = len(A[0])
    M = [[0 for i in range(2*n)] for j in range(2*n)]
    for i in range(n):
        for j in range(n):
            M[i][j] = A[i][j]
            M[i][j + n] = B[i][j]
            M[i + n][j] = C[i][j]
            M[i + n][j + n] = D[i][j]

    return M

def mat_mul(A, B):
    '''
    Conventional multiplication of two n by n matrices: A, B
    '''
    # initialize dimension of matrix and number of operations
    n = len(A[0])
    # initialize final matrix
    C = [[0 for i in range(n)] for j in range(n)]
    # implement contraction
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k]*B[k][j]
    return C

def strassen(M1, M2, top_level = 0):
    n0 = 512
    # for the highest level of recursion
    if top_level:
        global ops
        ops = 0
        
    n = len(M1[0])
    if n <= n0:
        return mat_mul(M1, M2)
    # base case
    if n == 1:
        ops += 1
        return [[M1[0][0]*M2[0][0]]]
    # if n is odd pad and unpad later before recursion
    if n % 2:
        M1 = pad(M1)
        M2 = pad(M2)
    
    # split into quadrants
    A, B, C, D = split(M1)
    E, F, G, H = split(M2)
    
    P1 = strassen(A, add(F, H, 0))
    P2 = strassen(add(A, B), H)
    P3 = strassen(add(C, D), E)
    P4 = strassen(D, add(G, E, 0))
    P5 = strassen(add(A, D), add(E, H))
    P6 = strassen(add(B, D, 0), add(G, H))
    P7 = strassen(add(A, C, 0), add(E, F))
    
    # calculate strass submatrices
    AEBG = add(add(add(P5, P4), P2, 0), P6)
    AFBH = add(P1, P2)
    CEDG = add(P3, P4)
    CFDH = add(add(add(P5, P1), P3, 0), P7, 0)
    
    M = merge(AEBG, AFBH, CEDG, CFDH)
    
    # if n is odd
    if n % 2:
        M = unpad(M)
        M1 = unpad(M1)
        M2 = unpad(M2)
    
    return M

def compare_methods(A, B):

    pass

In [2]:
n = 6

A = random_mat(n)
B = random_mat(n)
res = strassen(A, B, 1)

print(ops)
print_mat(res)

1891
2 1 0 1 2 1
2 2 0 2 3 1
2 2 1 3 4 1
2 1 1 2 2 1
1 2 0 1 3 1
1 0 1 1 3 0



In [31]:
from random import random
from time import time

def random_graph(p, dim):
    A = [[0 for i in range(dim)] for j in range(dim)]
    for i in range(dim):
        for j in range(i+1, dim):
            if random() < p:
                A[i][j] = 1
                A[j][i] = 1
    return A
G = lambda p: random_graph(p, 1024)
ps = [0.01, 0.02, 0.03, 0.04, 0.05]

for p in ps:
    print(f'p = {p}')
    A = G(p)
    print(f'Performing first multiplication...')
    B = strassen(A, A)
    print(f'Performing second multiplication...')
    C = strassen(A, B)
    print(f'Summing diagonal...')
    count = 0
    for i in range(len(C)):
        count += C[i][i]
    res = count / 6
    print(f'Number of triangles: {res}')
    print()

p = 0.01
Performing first multiplication...
Performing second multiplication...
Summing diagonal...
Number of triangles: 183.0

p = 0.02
Performing first multiplication...
Performing second multiplication...
Summing diagonal...
Number of triangles: 1392.0

p = 0.03
Performing first multiplication...
Performing second multiplication...
Summing diagonal...
Number of triangles: 4955.0

p = 0.04
Performing first multiplication...
Performing second multiplication...
Summing diagonal...
Number of triangles: 11120.0

p = 0.05
Performing first multiplication...
Performing second multiplication...
Summing diagonal...
Number of triangles: 22495.0



0.8379896963937611